In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.csv('dbfs:/FileStore/file-valid/sales_so_042022.csv', inferSchema=True, header=True)

In [0]:
display(df)

invoice_date,id,name,father,city,address,marks,phone,_c8
2022-04-04,1,sam,king,scranton,"11/497, Box st, Scranton, PA",67,-,null
2022-04-30,2,daniel,oliver,newyork,*#,34,76548263,null
2022-04-01,3,alex,joah,washington,-,55,65739302,null
2022-04-09,4,simon,nim,tokyo,"#456, North street, Tokyo",77,1417469493,2
2022-04-15,5,ross,james,paris,"Faux Ville, 1112, Paris",88,860485,fr
2022-04-16,6,joe,stephen,chicago,"Aswert, NY",51,675656573,null
2022-05-08,7,stephen,stephen,stephen,US,69,837-75846-8,null


In [0]:
df = df.withColumn('signal',when((col('name')==col('father'))|(col('name')==col('city'))|(col('father')==col('city')), False)
              .otherwise(True))
if False == df.filter(df.signal.contains(False)).collect()[0]['signal']:
    print(f'Unique Row Values Validation: name, father and city contain same values')

Unique Row Values Validation: name, father and city contain same values
